<a href="https://colab.research.google.com/github/sjkim-audio/audio-dsp-lab/blob/main/notebooks/09_Audio_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# [가정] 특정 형태 데이터셋을 가정해 모델 구조만 설계
# Input Shape: (Batch_Size, MFCC_coeffs, Time_steps, Channels)
# 예: (None, 13, 44, 1) - 13개 MFCC 특징, 약 1초 길이
input_shape = (13, 44, 1)
n_classes = 3 # 분류할 클래스 개수

# 1. CNN 모델 설계 (Model Architecture)
# 오디오 스펙트로그램(MFCC)을 '이미지'처럼 처리하여 패턴을 찾습니다.
model = models.Sequential([

    # [Layer 1] 특징 추출 (Feature Extraction)
    # Conv2D: 3x3 필터로 시간-주파수 패턴을 훑음
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
    layers.MaxPooling2D((2, 2)), # 중요 정보만 남기고 사이즈 압축
    layers.BatchNormalization(), # 학습 안정화

    # [Layer 2] 심화 특징 추출
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),

    # [Layer 3] 분류기 (Classifier)
    layers.Flatten(), # 2차원 데이터를 1차원으로 펼침
    layers.Dense(64, activation='relu'), # 판단을 위한 뉴런층
    layers.Dropout(0.3), # 과적합(Overfitting) 방지

    # [Output Layer] 최종 결과 출력
    layers.Dense(n_classes, activation='softmax')
    # Softmax: 각 클래스일 확률을 출력 (예: 개 80%, 고양이 10%, 사람 10%)
])

# 2. 모델 컴파일 (Compile)
# 최적화 도구(Adam)와 채점 기준(Loss Function) 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3. 모델 구조 확인
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 13, 44, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 22, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 6, 22, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 6, 22, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 3, 11, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 11, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2112)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       135,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 154,627 (604.01 KB)

 Trainable params: 154,435 (603.26 KB)

 Non-trainable params: 192 (768.00 B)